# BigML Submission

Note this notebook was intended to be run on Google Colab in a fresh environment.

In [1]:
!git clone https://github.com/BigML-CS-UCLA/SpuCo.git

Cloning into 'SpuCo'...
remote: Enumerating objects: 3353, done.
remote: Counting objects: 100% (1839/1839), done.
remote: Compressing objects: 100% (394/394), done.
remote: Total 3353 (delta 1477), reused 1684 (delta 1378), pack-reused 1514
Receiving objects: 100% (3353/3353), 89.89 MiB | 11.57 MiB/s, done.
Resolving deltas: 100% (2235/2235), done.


In [2]:
%pip install SpuCo

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [55]:
import torch

device = torch.device('cpu')

In [56]:
from spuco.utils import set_seed

set_seed(0)

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.99,
    classes=classes,
    split="train"
)
trainset.initialize()


valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16235692.23it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 492709.94it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4478563.23it/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3862637.63it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 10000/10000 [00:01<00:00, 7362.12it/s]


In [57]:
print(len(trainset))
print(len(valset))
print(len(testset))

48004
11996
10000


In [58]:
import torchvision.transforms as T

T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [60]:
from torch.optim import SGD
from spuco.robust_train import ERM
from spuco.models import model_factory

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
erm = ERM(
    model=model,
    num_epochs=1,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

In [61]:
erm.train()

Epoch 0: 100%|██████████| 751/751 [00:13<00:00, 56.53batch/s, accuracy=100.0%, loss=0.0075]


In [62]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.04it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.03it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:07,  3.09it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.85it/s]

Group (4, 3) Accuracy: 2.272727272727273


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:09<00:00,  2.74it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 2.272727272727273,
 (4, 4): 100.0}

In [63]:
from spuco.group_inference import Cluster, ClusterAlg

logits = erm.trainer.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=5,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 11.61it/s]


In [64]:
for key in sorted(group_partition.keys()):
    print(key, len(group_partition[key]))

(0, 0) 5570
(0, 1) 4461
(0, 2) 30
(0, 3) 28
(0, 4) 44
(1, 0) 9575
(1, 1) 32
(1, 2) 23
(1, 3) 24
(1, 4) 18
(2, 0) 5969
(2, 1) 2951
(2, 2) 32
(2, 3) 42
(2, 4) 17
(3, 0) 4248
(3, 1) 1917
(3, 2) 3484
(3, 3) 24
(3, 4) 74
(4, 0) 25
(4, 1) 7002
(4, 2) 44
(4, 3) 2345
(4, 4) 25


In [65]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=trainset,
    group_partition=group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:02<00:48,  2.04s/it]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:03<00:43,  1.91s/it]

Group (0, 1) Accuracy: 100.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:04<00:25,  1.17s/it]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:14<00:00,  1.50it/s]

Group (4, 3) Accuracy: 99.95735607675905


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:15<00:00,  1.66it/s]

Group (4, 4) Accuracy: 0.0


{(0, 0): 100.0,
 (0, 1): 100.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 100.0,
 (1, 1): 0.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 100.0,
 (2, 1): 100.0,
 (2, 2): 0.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 100.0,
 (3, 1): 100.0,
 (3, 2): 100.0,
 (3, 3): 0.0,
 (3, 4): 0.0,
 (4, 0): 0.0,
 (4, 1): 100.0,
 (4, 2): 0.0,
 (4, 3): 99.95735607675905,
 (4, 4): 0.0}

In [66]:
from torch.optim import SGD
from spuco.robust_train import GroupBalanceBatchERM, ClassBalanceBatchERM
from spuco.models import model_factory

model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=8,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

Epoch 7: 100%|██████████| 751/751 [00:13<00:00, 54.56batch/s, accuracy=100.0%, loss=0.00142]


In [67]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:08,  2.95it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.00it/s]

Group (0, 1) Accuracy: 96.92671394799055


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:07,  2.99it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  2.48it/s]

Group (4, 3) Accuracy: 70.95959595959596


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:09<00:00,  2.76it/s]

Group (4, 4) Accuracy: 98.23232323232324


{(0, 0): 100.0,
 (0, 1): 96.92671394799055,
 (0, 2): 88.65248226950355,
 (0, 3): 86.28841607565012,
 (0, 4): 90.78014184397163,
 (1, 0): 84.10757946210269,
 (1, 1): 94.62102689486552,
 (1, 2): 79.90196078431373,
 (1, 3): 79.41176470588235,
 (1, 4): 72.54901960784314,
 (2, 0): 80.53333333333333,
 (2, 1): 82.4,
 (2, 2): 98.13333333333334,
 (2, 3): 71.46666666666667,
 (2, 4): 71.12299465240642,
 (3, 0): 87.68844221105527,
 (3, 1): 78.58942065491183,
 (3, 2): 78.33753148614609,
 (3, 3): 98.48866498740554,
 (3, 4): 87.40554156171285,
 (4, 0): 86.39798488664988,
 (4, 1): 76.0705289672544,
 (4, 2): 72.29219143576826,
 (4, 3): 70.95959595959596,
 (4, 4): 98.23232323232324}

In [68]:
evaluator.worst_group_accuracy

((4, 3), 70.95959595959596)

In [69]:
evaluator.average_accuracy

97.7490318445806

In [70]:
evaluator.evaluate_spurious_attribute_prediction()

29.57